In [6]:
import os
import numpy as np
import librosa
import tensorflow as tf

def generate_uint8_mfcc_from_wav(
    wav_path,
    tflite_model_path,
    header_path,
    variable_name="g_mfcc_sample_uint8"):
    """
    Processes a .wav file to generate uint8 quantized MFCC features and
    exports them to a C header file.
    """
    try:
        # --- Load the TFLite model to get quantization parameters ---
        if not os.path.exists(tflite_model_path):
            print(f"Error: TFLite model not found at '{tflite_model_path}'")
            return

        interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()[0]

        # Check if the model input is uint8
        if 'quantization' not in input_details or input_details['dtype'] != np.uint8:
            print("Error: The provided TFLite model's input is not uint8 quantized.")
            print(f"Model expects: {input_details['dtype']}")
            return

        # Get the scale and zero point for the input tensor
        input_scale, input_zero_point = input_details['quantization']
        print(f"Model Input Quantization Params: scale={input_scale}, zero_point={input_zero_point}")


        # --- Preprocessing Parameters (Must match your original model) ---
        sample_rate = 16000
        n_fft = 640
        hop_length = 320
        n_mels = 40
        n_mfcc = 10
        fmin = 20
        fmax = 4000

        # --- Load and Preprocess Audio to Float MFCCs ---
        y, sr = librosa.load(wav_path, sr=sample_rate)
        if len(y) > sample_rate: y = y[:sample_rate]
        else: y = np.pad(y, (0, max(0, sample_rate - len(y))), 'constant')

        mel_spec = librosa.feature.melspectrogram(
            y=y, sr=sample_rate, n_fft=n_fft, hop_length=hop_length,
            n_mels=n_mels, fmin=fmin, fmax=fmax, center=False
        )
        db_spec = librosa.power_to_db(mel_spec)
        mfcc_features_float = librosa.feature.mfcc(S=db_spec, n_mfcc=n_mfcc).T


        # --- Quantize the Float MFCCs to uint8 ---
        # Formula: uint_val = (float_val / scale) + zero_point
        mfcc_features_uint8 = (mfcc_features_float / input_scale) + input_zero_point

        # ** FIX **: Clip to the valid uint8 range [0, 255] and cast to uint8
        mfcc_features_uint8 = np.clip(mfcc_features_uint8, 0, 255).astype(np.uint8)

        mfcc_flat_uint8 = mfcc_features_uint8.flatten()

        # --- Write to C header file ---
        with open(header_path, 'w') as f:
            f.write(f"/*\n * UINT8 Quantized MFCC Feature data from: {os.path.basename(wav_path)}\n */\n\n")
            f.write("#ifndef MFCC_SAMPLE_UINT8_H\n")
            f.write("#define MFCC_SAMPLE_UINT8_H\n\n")
            f.write("#include <stdint.h>\n\n")
            f.write(f"#define {variable_name.upper()}_LEN {len(mfcc_flat_uint8)}\n\n")
            # ** FIX **: Use uint8_t for the C array type
            f.write(f"const uint8_t {variable_name}[{len(mfcc_flat_uint8)}] = {{\n    ")

            for i, val in enumerate(mfcc_flat_uint8):
                f.write(f"{val}, ")
                if (i + 1) % 16 == 0:
                    f.write("\n    ")

            f.write("\n};\n\n")
            f.write("#endif /* MFCC_SAMPLE_UINT8_H */\n")

        print(f"Successfully generated uint8 MFCC features into '{header_path}'")

    except Exception as e:
        print(f"An error occurred: {e}")

# --- USAGE ---
# 1. Provide the path to the audio file you want to test
input_wav_file = 'right.wav'

# 2. !! IMPORTANT !! Provide the path to your FULLY QUANTIZED .tflite model
quantized_tflite_file = 'ds_cnn_int8.tflite'

# 3. Define the output header file
output_header_file = 'mfcc_right_sample_uint8.h'
c_variable_name = 'g_right_mfcc_sample_uint8'

if not os.path.exists(input_wav_file):
    print(f"Error: Input WAV file '{input_wav_file}' not found.")
else:
    generate_uint8_mfcc_from_wav(input_wav_file, quantized_tflite_file, output_header_file, c_variable_name)


Model Input Quantization Params: scale=2.602257490158081, zero_point=198
Successfully generated uint8 MFCC features into 'mfcc_right_sample_uint8.h'
